In [ ]:
from pathlib import Path

In [ ]:
__file__ = "openwrt.ipynb"
TopDir = Path(__file__).parent.absolute()
print(TopDir)
ProjectName = 'ctcgfw'


In [ ]:
# internal var
ProjectDir = TopDir.joinpath(ProjectName)
PackageInfoFile:Path = ProjectDir.joinpath('tmp', '.packageinfo')
TargetInfoFile:Path = ProjectDir.joinpath('tmp', '.targetinfo')
print(PackageInfoFile)
print(TargetInfoFile)

In [ ]:
import re
from src.lexer.packageinfo import PackageInfoLexer
from src.parser.packageinfo import PackageInfoParser
import ply.yacc as yacc
import json

testFullString = PackageInfoFile.read_text()
### hack some code to pass through laxer
testFullString = re.sub('\t\t\t/sys/module/iwlwifi/parameters/debug','\t\t  /sys/module/iwlwifi/parameters/debug',testFullString)
testFullString = re.sub('\t\t\t  % echo 0x43fff > /sys/module/iwlwifi/parameters/debug','\t\t    % echo 0x43fff > /sys/module/iwlwifi/parameters/debug',testFullString)
testFullString = re.sub('\t\t\t  drivers/net/wireless/intel/iwlwifi/iwl-debug.h','\t\t    drivers/net/wireless/intel/iwlwifi/iwl-debug.h',testFullString)

m = PackageInfoLexer()
m.build(debug=False)           # Build the lexer
p = PackageInfoParser()
p.build(m.lexer)
packageInfoAst = p.test(testFullString)
# with open('ast.json','w') as f:
#     f.write(json.dumps(packageInfoAst,indent=4,sort_keys=True))
# print(p.test(testFullString))
# m.testSlient(testFullString)     # Test it
print('Done')


In [ ]:
from src.distributor import DependsTree
tree = DependsTree()
tree.build(packageInfoAst)


In [ ]:
from src.distributor import WorksDistributor
distributor = WorksDistributor()
distributor.build(tree)
distributor.divide(20)
